## Word list to vector function

In [1]:
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', \
                          'problems', 'help', 'please'],
                         ['maybe', 'not', 'take', 'him', \
                          'to', 'dog', 'park', 'stupid'],
                         ['my', 'dalmation', 'is', 'so', 'cute', \
                           'I', 'love', 'him'],
                         ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                         ['mr', 'licks', 'ate', 'my', 'steak', 'how',\
                           'to', 'stop', 'him'],
                         ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]  # 1 is abusive, 0 not
    return postingList, classVec

In [2]:
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

In [3]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: print("the word: {} is not in my Vocabulary".format(word))
    return returnVec

In [4]:
listOPosts,listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
print(myVocabList)
print([0]*3)
returnVec = setOfWords2Vec(myVocabList, listOPosts[0])
print(returnVec)

['steak', 'garbage', 'dog', 'please', 'dalmation', 'maybe', 'licks', 'buying', 'to', 'cute', 'not', 'mr', 'love', 'stupid', 'my', 'park', 'is', 'how', 'posting', 'help', 'flea', 'quit', 'worthless', 'so', 'stop', 'ate', 'problems', 'food', 'him', 'has', 'take', 'I']
[0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]


## Naive Bayes classifier training function

In [5]:
def trainNB0(trainMatrix, trainCategory):
    numTrainDOcs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDOcs) # we only have two classes:0 and 1
    p0Num = ones(numWords); p1Num=ones(numWords)  # deal with zero value effect
    p0Denom = 2.0; p1Denom=2.0
    for i in range(numTrainDOcs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num/p1Denom)   #change to log()
    p0Vect = log(p0Num/p0Denom)   #change to log()
    return p0Vect, p1Vect, pAbusive
            

In [6]:
from numpy import *
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
trainMat=[]
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
p0V,p1V,pAb=trainNB0(trainMat, listClasses)
print(pAb)
print(p0V)
print(p1V)

0.5
[-2.56494936 -3.25809654 -2.56494936 -2.56494936 -2.56494936 -3.25809654
 -2.56494936 -3.25809654 -2.56494936 -2.56494936 -3.25809654 -2.56494936
 -2.56494936 -3.25809654 -1.87180218 -3.25809654 -2.56494936 -2.56494936
 -3.25809654 -2.56494936 -2.56494936 -3.25809654 -3.25809654 -2.56494936
 -2.56494936 -2.56494936 -2.56494936 -3.25809654 -2.15948425 -2.56494936
 -3.25809654 -2.56494936]
[-3.04452244 -2.35137526 -1.94591015 -3.04452244 -3.04452244 -2.35137526
 -3.04452244 -2.35137526 -2.35137526 -3.04452244 -2.35137526 -3.04452244
 -3.04452244 -1.65822808 -3.04452244 -2.35137526 -3.04452244 -3.04452244
 -2.35137526 -3.04452244 -3.04452244 -2.35137526 -1.94591015 -3.04452244
 -2.35137526 -3.04452244 -3.04452244 -2.35137526 -2.35137526 -3.04452244
 -2.35137526 -3.04452244]


## Naive Bayes classify function

In [7]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0: 
        return 1
    else:
        return 0

In [8]:
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat), array(listClasses))
    testEntry=['love', 'my','dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ', classifyNB(thisDoc, p0V,p1V,pAb))
    testEntry=['stupid','garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ', classifyNB(thisDoc, p0V,p1V,pAb))
    

In [9]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


## Naive Bayes bag-of-words model

In [10]:
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

## Example: classifying spam email with naive Bayes

In [11]:
def textParse(bigString):
    import re
    listOfTokens = re.split(r'\W*', str(bigString))
    return [tok.lower() for tok in listOfTokens if len(tok)>2]
def spamTest():
    docList = []; classList = []; fullText=[]
    for i in range(1,26):
        wordList = textParse(open('email/spam/%d.txt' % i,'rb').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i,'rb').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    trainingSet = list(range(50)); testSet=[]
    for i in range(10):
        randIndex=int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del (trainingSet[randIndex])
    trainMat=[]; trainClasses=[]
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam=trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(array(wordVector), p0V,p1V,pSpam)!=classList[docIndex]:
            errorCount += 1
    errorRate = float(errorCount)/len(testSet)
    print('the error rate is: {}'.format(errorRate))
    return errorRate


In [12]:
errorRateAll = 0.0
for i in range(10):
    errorRateAll += spamTest()
print('the average error rate is: {}'.format(errorRateAll/10.0))

the error rate is: 0.8
the error rate is: 0.8
the error rate is: 0.5
the error rate is: 0.6
the error rate is: 0.5
the error rate is: 0.5
the error rate is: 0.7
the error rate is: 0.5
the error rate is: 0.8
the error rate is: 0.8
the average error rate is: 0.65


## Example: using naïve Bayes to reveal local attitudes from personal ad

In [ ]:
def calcMostFreq(vocabList, fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token] = fullText.count(token)
    sortedFreq = sorted(freqDict.iteritems(), key=operator.itemgetter(1),
                       reverser=True)
    return sortedFreq[:30]

def localWords(feed1, feed0):
    import feedparser
    docList = []; classList = []; fullText = []
    minLen = min(len(feed1['entries']), len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    top30Words = calcMostFreq(vocabList, fullText)
    for pairW in top30Words:
        if pairW[0] in vocabList: vocabList.remove(pairW[0])
    trainingSet = range(2*minLen); testSet=[]
    for i in range(20):
        randIndex = int(random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat), array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam)!=classList[docIndex]:
            errorCount += 1
    errorRate = float(errorCount)/len(testSet)
    print('the error rate is: {}'.format(errorRate))
    return vocabList,p0V,p1V